# Data Prepration

## how to define holiday

any day in which consecusitively there are zero sales for more than 12 hours can be treated as store closed we have to augment the data for this as given data is for the number of days and then we have converted days into hours so we can just randomly fill every 7th day as store closed and can mark all the sales to zero and we can select some random days also to flag the as zero

## how to define promotion day

sales greater than 90th percentile can be treated as promotion days we can see first the distribution of data that what are the unit sales value for 95th percentile of the data and if there are more number of days then we can push this to 97 percentile

In [18]:
import os
import pandas as pd

In [19]:
abs_data_path = r"D:\ds_projects\kaggle\m5\data"

In [20]:
sell_price_path = os.path.join(abs_data_path, "sell_prices.csv") 
sales_data_path = os.path.join(abs_data_path, "sales_train_validation.csv") 
calendar_data_path = os.path.join(abs_data_path, "calendar.csv")

In [21]:
calendar = pd.read_csv(calendar_data_path, keep_default_na=False)
sales_train_validation = pd.read_csv(sales_data_path, keep_default_na=False)
# sample_submission = pd.read_csv("sample_submission.csv", keep_default_na=False)
# sell_prices = pd.read_csv(sell_price_path, keep_default_na=False)

In [22]:
sales_train_validation.describe()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
count,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,...,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000,30490.000000
mean,1.070220,1.041292,0.780026,0.833454,0.627944,0.958052,0.918662,1.244080,1.073663,0.838701,...,1.370581,1.586159,1.693670,1.248245,1.232207,1.159167,1.149000,1.328862,1.605838,1.633158
std,5.126689,5.365468,3.667454,4.415141,3.379344,4.785947,5.059495,6.617729,5.917204,4.206199,...,3.740017,4.097191,4.359809,3.276925,3.125471,2.876026,2.950364,3.358012,4.089422,3.812248
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
max,360.000000,436.000000,207.000000,323.000000,296.000000,314.000000,316.000000,370.000000,385.000000,353.000000,...,129.000000,160.000000,204.000000,98.000000,100.000000,88.000000,77.000000,141.000000,171.000000,130.000000


In [8]:
x = list(sales_train_validation.columns[6:])
change_col = [
    [
        "d_{}".format(int(x/12)+1), #date_id
        int(x/12)+1, #date
        int(x%12)+8, #hour
        int(x/12)%7 #day
    ] for x in range(len(x))
]
change_col[:5], change_col[-1:-5:-1],# change_col

([['d_1', 1, 8, 0],
  ['d_1', 1, 9, 0],
  ['d_1', 1, 10, 0],
  ['d_1', 1, 11, 0],
  ['d_1', 1, 12, 0]],
 [['d_160', 160, 12, 5],
  ['d_160', 160, 11, 5],
  ['d_160', 160, 10, 5],
  ['d_160', 160, 9, 5]])

In [9]:
date_df = pd.DataFrame(change_col, columns=["date_id", "date", "hour", "day"])

In [10]:
sales_train = pd.melt(
    sales_train_validation,
    id_vars=sales_train_validation.columns[:6],
    value_vars=sales_train_validation.columns[6:],
    var_name='date_id',
    value_name='quantity'
).merge(
    date_df,
    on="date_id"
)

In [11]:
sales_train = sales_train[
    list(sales_train_validation.columns[:6])+list(date_df.columns)+["quantity"]
]

In [12]:
len(sales_train)

58327370

In [13]:
sales_train.to_csv(os.path.join(abs_data_path, "sales_train.csv"), index=False)

In [14]:
sales_train

,id,item_id,dept_id,cat_id,store_id,state_id,date_id,date,hour,day,quantity
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,8,0,0
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,9,0,0
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,10,0,0
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,11,0,0
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,1,12,0,0
...,...,...,...,...,...,...,...,...,...,...,...
58327365,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_160,160,8,5,0
58327366,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_160,160,9,5,0
58327367,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_160,160,10,5,0
58327368,FOODS_3_827_WI_3_validation,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,d_160,160,11,5,0


# logic

## for store closure 
we can use the logic that any store showing continous 0 sales for more than we can flag them as store being closed

## to identify the run rate for the particular product 

we can run the univariate for first store product combination at row level and then at product level for the same day and  can see the results this will give us the matrix of 12 hours * 7 week days combination 

run rate will be useful measure to trigger the msg if the shelf is getting emptied 

# for inventory management
